# Задание

Дан датасет с несправедливо осужденными заключенными. Требуется произвести его кластеризацию.
Для этого требуется:
1. Проанализировать данные, выбрать нужные, удалить ненужные столбцы. При необходимости использовать label encoding или one hot encoding для преобразования категориальных признаков в численные.
2. Выбрать оптимальное количество кластеров для метода K-means++ при помощи метода локтя.
3. Использовать метод K-means++ для кластеризации датасета на оптимальном количестве кластеров, найденном на предыдущем шаге. 
4. Проанализировать полученные кластеры и интерпретировать результаты.

Ссылка на библиотеку sklearn с реализацией K-means++:
[ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

## Установка библиотек и имопрты

In [8]:
# %pip install pandas numpy plotly nbformat scikit-learn xgboost

In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import abc as abc
import sklearn as skl

## Загрузка дата сета

In [10]:
df = pd.read_csv("./data/wrong_conv_in_us.csv")


df_clean = df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          178 non-null    int64 
 1   Date of the Crime           178 non-null    int64 
 2   Defendants                  178 non-null    object
 3   Type of the Crime           178 non-null    object
 4   Location of the Crime       178 non-null    object
 5   Punishment for a crime      169 non-null    object
 6   Legally Exonerated          178 non-null    object
 7   Legally Exonerated Encoded  178 non-null    int64 
dtypes: int64(3), object(5)
memory usage: 11.3+ KB


In [11]:
df.describe()

,ID,Date of the Crime,Legally Exonerated Encoded
count,178.000000,178.000000,178.000000
mean,88.500000,1980.415730,0.859551
std,51.528309,34.544758,0.348433
min,0.000000,1805.000000,0.000000
25%,44.250000,1978.000000,1.000000
50%,88.500000,1986.500000,1.000000
75%,132.750000,1998.000000,1.000000
max,177.000000,2017.000000,1.000000


In [12]:
df.head()

,ID,Date of the Crime,Defendants,Type of the Crime,Location of the Crime,Punishment for a crime,Legally Exonerated,Legally Exonerated Encoded
0,0,1805,Dominic Daley and James Halligan,murder,Massachusetts,death,yes,1
1,1,1843,John Gordon,murder,Rhode Island,death,yes,1
2,2,1855,Chief Leschi,murder,Washington,death,no,0
3,3,1863,Chipita Rodriguez,murder,Texas,death,yes,1
4,4,1872,William Jackson Marion,murder,Nebraska,death,yes,1


## Анализ и подготовка данных

In [13]:
print("Пропущенные значения:")
print(df_clean.isnull().sum())
print("\nТипы данных:")
print(df_clean.dtypes)

Пропущенные значения:
ID                            0
Date of the Crime             0
Defendants                    0
Type of the Crime             0
Location of the Crime         0
Punishment for a crime        9
Legally Exonerated            0
Legally Exonerated Encoded    0
dtype: int64

Типы данных:
ID                             int64
Date of the Crime              int64
Defendants                    object
Type of the Crime             object
Location of the Crime         object
Punishment for a crime        object
Legally Exonerated            object
Legally Exonerated Encoded     int64
dtype: object


In [14]:
df_clean = df_clean.drop("Legally Exonerated Encoded", axis=1)

df_clean = df_clean.dropna()

print(f"Размер датасета после очистки: {df_clean.shape}")

Размер датасета после очистки: (169, 7)


In [15]:
# Кодируем категориальные переменные
from sklearn.preprocessing import LabelEncoder

le_crime = LabelEncoder()
le_location = LabelEncoder()
le_punishment = LabelEncoder()
le_exonerated = LabelEncoder()
le_defendants = LabelEncoder()

df_clean["Type of the Crime"] = le_crime.fit_transform(
    df_clean["Type of the Crime"]
)
df_clean["Location of the Crime"] = le_location.fit_transform(
    df_clean["Location of the Crime"]
)
df_clean["Punishment for a crime"] = le_punishment.fit_transform(
    df_clean["Punishment for a crime"]
)
df_clean["Legally Exonerated"] = le_exonerated.fit_transform(
    df_clean["Legally Exonerated"]
)
df_clean["Defendants"] = le_defendants.fit_transform(df_clean["Defendants"])

print(df_clean.head())
print("\nИтоговые типы данных:")
print(df_clean.dtypes)

   ID  Date of the Crime  Defendants  Type of the Crime  \
0   0               1805          43                 16   
1   1               1843          74                 16   
2   2               1855          21                 16   
3   3               1863          22                 16   
4   4               1872         166                 16   

   Location of the Crime  Punishment for a crime  Legally Exonerated  
0                     16                      35                   1  
1                     30                      35                   1  
2                     35                      35                   0  
3                     33                      35                   1  
4                     21                      35                   1  

Итоговые типы данных:
ID                        int64
Date of the Crime         int64
Defendants                int64
Type of the Crime         int64
Location of the Crime     int64
Punishment for a crime    int64
Lega

In [16]:
# Нормализуем данные
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(df_clean)

print(f"Размер матрицы признаков: {X.shape}")

Размер матрицы признаков: (169, 7)


## Метод локтя для поиска оптимального k

In [17]:
from sklearn.cluster import KMeans

inertias = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, init="k-means++", random_state=42, n_init=10)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=list(k_range), y=inertias, mode="lines+markers", name="Inertia"
    )
)
fig.update_layout(
    title="Метод локтя",
    xaxis_title="Количество кластеров (k)",
    yaxis_title="Inertia",
)
fig.show()

In [18]:
optimal_k = 5

kmeans = KMeans(n_clusters=optimal_k, init="k-means++", random_state=42)
clusters = kmeans.fit_predict(X)

print(f"Оптимальное количество кластеров: {optimal_k}")
print(f"Распределение по кластерам: {np.bincount(clusters)}")

Оптимальное количество кластеров: 5
Распределение по кластерам: [65 23 18 40 23]


## Анализ кластеров

In [19]:
df_clean["Cluster"] = clusters

print("Средние значения признаков по кластерам:")
cluster_analysis = df_clean.groupby("Cluster").mean()
print(cluster_analysis)

Средние значения признаков по кластерам:
                 ID  Date of the Crime  Defendants  Type of the Crime  \
Cluster                                                                 
0         73.938462        1984.061538   71.846154          16.969231   
1        141.826087        2004.086957   98.869565          11.347826   
2          9.666667        1896.111111   82.277778          17.444444   
3        103.425000        1991.800000   94.800000          16.700000   
4        118.347826        1997.130435   86.043478          33.521739   

         Location of the Crime  Punishment for a crime  Legally Exonerated  
Cluster                                                                     
0                    13.061538               32.907692            0.676923  
1                    20.739130               12.391304            0.956522  
2                    15.944444               33.555556            0.944444  
3                    27.150000               36.050000        

In [20]:
clusters

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 0,
       0, 0, 0, 4, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
       4, 0, 3, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 3, 3, 4, 0, 0, 0, 0, 1,
       4, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3, 4, 4, 3, 3, 1, 0, 4, 0, 3, 3, 0,
       4, 1, 3, 3, 0, 4, 3, 3, 0, 3, 3, 4, 1, 0, 3, 0, 0, 0, 3, 0, 0, 0,
       1, 0, 3, 1, 3, 0, 0, 3, 0, 4, 4, 3, 3, 0, 3, 4, 1, 0, 3, 1, 3, 4,
       1, 0, 3, 3, 3, 1, 1, 1, 4, 1, 3, 0, 3, 4, 4, 4, 4, 4, 0, 0, 1, 1,
       1, 1, 1, 3, 4, 1, 3, 1, 1, 4, 0, 4, 1, 1, 3], dtype=int32)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

fig = px.scatter(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    color=clusters,
    title="Кластеры (PCA визуализация)",
    labels={
        "x": f"PC1 ({pca.explained_variance_ratio_[0]:.2%})",
        "y": f"PC2 ({pca.explained_variance_ratio_[1]:.2%})",
    },
)
fig.show()

In [22]:
print("Анализ кластеров:\n")

for cluster_id in range(optimal_k):
    cluster_data = df_clean[df_clean["Cluster"] == cluster_id]
    print(f"Кластер {cluster_id}:")
    print(f"  Размер: {len(cluster_data)} образцов")
    print(f"  Средний год: {cluster_data['Date of the Crime'].mean():.0f}")
    print(
        f"  Всего реабилитирован (без ошибок): {(cluster_data['Legally Exonerated'] == 1).sum()}"
    )
    print()

Анализ кластеров:

Кластер 0:
  Размер: 65 образцов
  Средний год: 1984
  Всего реабилитирован (без ошибок): 44

Кластер 1:
  Размер: 23 образцов
  Средний год: 2004
  Всего реабилитирован (без ошибок): 22

Кластер 2:
  Размер: 18 образцов
  Средний год: 1896
  Всего реабилитирован (без ошибок): 17

Кластер 3:
  Размер: 40 образцов
  Средний год: 1992
  Всего реабилитирован (без ошибок): 39

Кластер 4:
  Размер: 23 образцов
  Средний год: 1997
  Всего реабилитирован (без ошибок): 22

